In [50]:
import cv2
import mediapipe as mp
import numpy as np
import os
from pathlib import Path


## Extracting Landmarks From Dataset

In [51]:
def extract_landmarks_from_video(video_path):
    mp_pose = mp.solutions.pose
    pose = mp_pose.Pose(static_image_mode=False)
    
    cap = cv2.VideoCapture(video_path)
    landmarks_all_frames = []

    while cap.isOpened():
        success, frame = cap.read()
        if not success:
            break

        # Convert frame to RGB
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        result = pose.process(frame_rgb)

        if result.pose_landmarks:
            # Extract (x, y, z) for 33 landmarks
            landmarks = []
            for lm in result.pose_landmarks.landmark:
                landmarks.extend([lm.x, lm.y, lm.z])
            landmarks_all_frames.append(landmarks)

    cap.release()
    pose.close()

    if landmarks_all_frames:
        return np.array(landmarks_all_frames)  # shape: (num_frames, 99)
    else:
        return None


In [52]:
def process_dataset_split(split_dir, label_map, output_dir="landmark_data"):
    Path(output_dir).mkdir(exist_ok=True)

    for class_name, label in label_map.items():
        class_path = os.path.join(split_dir, class_name)
        for video_file in os.listdir(class_path):
            if not video_file.endswith(".mp4"):
                continue

            video_path = os.path.join(class_path, video_file)
            landmarks = extract_landmarks_from_video(video_path)

            if landmarks is not None:
                output_filename = f"{split_dir.split('/')[-1]}_{class_name.replace(' ', '_')}_{video_file.replace('.mp4', '')}.npy"
                np.save(os.path.join(output_dir, output_filename), {
                    "landmarks": landmarks,
                    "label": label
                })


In [53]:
import cv2
import mediapipe as mp

# Initialize MediaPipe pose and drawing utils
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

# Load your video
video_path = "split_dataset/train/Arm Raise Correct/13.mp4"
cap = cv2.VideoCapture(video_path)

# Initialize pose estimator
with mp_pose.Pose(static_image_mode=False) as pose:
    while cap.isOpened():
        success, frame = cap.read()
        if not success:
            break

        # Convert frame to RGB
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Get pose landmarks
        result = pose.process(frame_rgb)

        # Draw the pose landmarks if detected
        if result.pose_landmarks:
            # Draw points and skeleton on original frame
            mp_drawing.draw_landmarks(
                frame,                         # frame to draw on
                result.pose_landmarks,         # detected landmarks
                mp_pose.POSE_CONNECTIONS       # predefined connections between keypoints
            )

        # Show the frame
        cv2.imshow('Pose Visualization', frame)

        # Exit on pressing 'q'
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

# Cleanup
cap.release()
cv2.destroyAllWindows()


In [54]:
label_map = {
    "Arm Raise Correct": 1,
    "Arm Raise Incorrect": 0
}

process_dataset_split("split_dataset/train", label_map)
process_dataset_split("split_dataset/val", label_map)
process_dataset_split("split_dataset/test", label_map)


In [55]:
import numpy as np
import os

# Paths
data_dir = "landmark_data"
X_train, y_train = [], []
X_val, y_val = [], []
X_test, y_test = [], []

# Loop over all .npy files
for file in os.listdir(data_dir):
    if not file.endswith(".npy"):
        continue

    data = np.load(os.path.join(data_dir, file), allow_pickle=True).item()
    landmarks = data['landmarks']    # shape: (num_frames, 99)
    label = data['label']            # 0 or 1

    # Convert to fixed size vector: average over frames → shape (99,)
    feature_vector = np.mean(landmarks, axis=0)

    # Decide which split it belongs to (based on filename prefix)
    if file.startswith("train"):
        X_train.append(feature_vector)
        y_train.append(label)
    elif file.startswith("val"):
        X_val.append(feature_vector)
        y_val.append(label)
    elif file.startswith("test"):
        X_test.append(feature_vector)
        y_test.append(label)

# Convert to numpy arrays
X_train = np.array(X_train)
y_train = np.array(y_train)
X_val = np.array(X_val)
y_val = np.array(y_val)
X_test = np.array(X_test)
y_test = np.array(y_test)

# Print shapes
print("Train:", X_train.shape, y_train.shape)
print("Val:", X_val.shape, y_val.shape)
print("Test:", X_test.shape, y_test.shape)


Train: (97, 99) (97,)
Val: (31, 99) (31,)
Test: (33, 99) (33,)


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import numpy as np
import joblib  # For saving the model

#  Print label distribution
print("Train Labels Count:", np.bincount(y_train))
print("Val Labels Count:", np.bincount(y_val))
print("Test Labels Count:", np.bincount(y_test))

# Initialize the model with tuned parameters
model = RandomForestClassifier(
    n_estimators=50,     # Fewer trees
    max_depth=3,         # Limit depth to reduce overfitting
    random_state=42
)

# Train the model
model.fit(X_train, y_train)

# Evaluate on validation set
val_preds = model.predict(X_val)
val_acc = accuracy_score(y_val, val_preds)
print("\n Validation Accuracy:", val_acc)

print("\nValidation Confusion Matrix:")
print(confusion_matrix(y_val, val_preds))

print("\nClassification Report (Validation):")
print(classification_report(y_val, val_preds, target_names=["Incorrect", "Correct"]))

#  Evaluate on test set
test_preds = model.predict(X_test)
test_acc = accuracy_score(y_test, test_preds)
print("\nTest Accuracy:", test_acc)

print("\nTest Confusion Matrix:")
print(confusion_matrix(y_test, test_preds))

print("\nClassification Report (Test):")
print(classification_report(y_test, test_preds, target_names=["Incorrect", "Correct"]))

# Show predictions vs actual for test set
print("\nPredictions vs Actual (Test Set):")
for i, (p, a) in enumerate(zip(test_preds, y_test)):
    print(f"Sample {i + 1}: Predicted = {p}, Actual = {a}")

# Save the trained model
joblib.dump(model, "posture_model.pkl")
print("\n Model saved as 'posture_model.pkl'")


Train Labels Count: [48 49]
Val Labels Count: [15 16]
Test Labels Count: [17 16]

 Validation Accuracy: 0.9354838709677419

Validation Confusion Matrix:
[[14  1]
 [ 1 15]]

Classification Report (Validation):
              precision    recall  f1-score   support

   Incorrect       0.93      0.93      0.93        15
     Correct       0.94      0.94      0.94        16

    accuracy                           0.94        31
   macro avg       0.94      0.94      0.94        31
weighted avg       0.94      0.94      0.94        31


Test Accuracy: 0.9393939393939394

Test Confusion Matrix:
[[17  0]
 [ 2 14]]

Classification Report (Test):
              precision    recall  f1-score   support

   Incorrect       0.89      1.00      0.94        17
     Correct       1.00      0.88      0.93        16

    accuracy                           0.94        33
   macro avg       0.95      0.94      0.94        33
weighted avg       0.95      0.94      0.94        33


Predictions vs Actual (Test

In [59]:
import cv2
import mediapipe as mp
import numpy as np
import joblib

# Load the trained model 
model = joblib.load("posture_model.pkl")

print(" Model loaded successfully")

# Initialize MediaPipe Pose 
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
pose = mp_pose.Pose(static_image_mode=False)

# Start Webcam 
cap = cv2.VideoCapture(0)

while cap.isOpened():
    success, frame = cap.read()
    if not success:
        break

    # Flip for mirror view
    frame = cv2.flip(frame, 1)

    # Convert frame to RGB
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    result = pose.process(frame_rgb)

    label_text = "No pose detected"

    if result.pose_landmarks:
        landmarks = []
        for lm in result.pose_landmarks.landmark:
            landmarks.extend([lm.x, lm.y, lm.z])

        if len(landmarks) == 99:
            # Convert to array and predict
            features = np.array(landmarks).reshape(1, -1)
            prediction = model.predict(features)[0]
            label_text = "Correct" if prediction == 1 else "Incorrect"

        # Draw pose skeleton on the frame
        mp_drawing.draw_landmarks(
            frame,
            result.pose_landmarks,
            mp_pose.POSE_CONNECTIONS
        )

    # Display prediction label
    cv2.putText(
        frame, label_text, (10, 40),
        cv2.FONT_HERSHEY_SIMPLEX, 1.2,
        (0, 255, 0) if label_text == "Correct" else (0, 0, 255),
        3
    )

    # Show the output
    cv2.imshow("Posture Classification", frame)

    # Press 'q' to quit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Cleanup 
cap.release()
cv2.destroyAllWindows()
pose.close()


 Model loaded successfully
